# Introduction to atomman: Defect creation and analysis

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction

One of the focuses of the atomman package is to provide tools for generating and analyzing atomic systems containing defects.  Many of these tools are more complex than the basic tools outlined in the previous section. This Notebook provides an overview of the various defect capabilities in atomman.

**Library Imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# https://matplotlib.org/
import matplotlib.pyplot as plt
%matplotlib inline

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.3.0
Notebook executed on 2019-11-05


## 2. Point defects

The atomman.defect.point() function inserts a point defect into a System and helps with tracking the defect by (1) moving any new/modified atoms to the end of the Atoms list and (2) adding per-atom property old_id that retains each atom's original index.

See [04.1. Point defects Jupyter Notebook](04.1._Point_defects.html) for more examples.

Parameters:

- __system__ (*atomman.System*) the system to add the defect to.

- __ptd_type__ (*str*) indicates which type of point defect to add.
    
    - ='v' -- vacancy
    
    - ='s' -- substitutional
    
    - ='i' -- positional interstitial
    
    - ='db' -- dumbbell interstitial

- __atype__ (*int, optional*) atom type for defect atom ('i', 's', 'db' styles).

- __pos__ (*array-like object, optional*) position for adding the defect atom (all styles).

- __ptd_id__ (*int, optional*) atom id where defect is added.  Alternative to using pos ('v', 's', 'db' styles).

- __db_vect__ (*array-like object, optional*) vector associated with the dumbbell interstitial ('db' style).

- __scale__ (*bool, optional*) indicates if pos and db_vect are absolute (False) or box-relative (True). Default is False.

- __atol__ (*float, optional*) absolute tolerance for position-based searching. Default is 1e-3 angstroms.

In [2]:
# Define 2x2x2 bcc demonstration supercell
a = uc.set_in_units(2.8665, 'angstrom')
box = am.Box(a=a, b=a, c=a)
atoms = am.Atoms(atype=1, pos=[[0.0,0.0,0.0], [0.5,0.5,0.5]])
ucell = am.System(atoms=atoms, box=box, scale=True)
system = ucell.supersize(2,2,2)
print(system.atoms_df())

    atype   pos[0]   pos[1]   pos[2]
0       1  0.00000  0.00000  0.00000
1       1  1.43325  1.43325  1.43325
2       1  2.86650  0.00000  0.00000
3       1  4.29975  1.43325  1.43325
4       1  0.00000  2.86650  0.00000
5       1  1.43325  4.29975  1.43325
6       1  2.86650  2.86650  0.00000
7       1  4.29975  4.29975  1.43325
8       1  0.00000  0.00000  2.86650
9       1  1.43325  1.43325  4.29975
10      1  2.86650  0.00000  2.86650
11      1  4.29975  1.43325  4.29975
12      1  0.00000  2.86650  2.86650
13      1  1.43325  4.29975  4.29975
14      1  2.86650  2.86650  2.86650
15      1  4.29975  4.29975  4.29975


In [3]:
# Change atom 5 into <111> dumbbell interstitial pair
defect_system = am.defect.point(system, 'db', ptd_id=5, db_vect=[0.3, 0.3, 0.3])
print(defect_system.atoms_df())

    atype   pos[0]   pos[1]   pos[2]  old_id
0       1  0.00000  0.00000  0.00000       0
1       1  1.43325  1.43325  1.43325       1
2       1  2.86650  0.00000  0.00000       2
3       1  4.29975  1.43325  1.43325       3
4       1  0.00000  2.86650  0.00000       4
5       1  2.86650  2.86650  0.00000       6
6       1  4.29975  4.29975  1.43325       7
7       1  0.00000  0.00000  2.86650       8
8       1  1.43325  1.43325  4.29975       9
9       1  2.86650  0.00000  2.86650      10
10      1  4.29975  1.43325  4.29975      11
11      1  0.00000  2.86650  2.86650      12
12      1  1.43325  4.29975  4.29975      13
13      1  2.86650  2.86650  2.86650      14
14      1  4.29975  4.29975  4.29975      15
15      1  1.13325  3.99975  1.13325       5
16      1  1.73325  4.59975  1.73325      16


## 3. Volterra dislocation solutions

*Updated version 1.2.5:* Ability to solve isotropic solutions added.

The atomman.defect.solve_volterra_dislocation() function generates an elasticity solution for a perfectly straight dislocation. The function returns either a Stroh object or a IsotropicVolterraDislocation object which has methods for obtaining position-based displacements and stress states, as well as energy-related parameters.  

See the [04.2. Volterra dislocation models Jupyter Notebook](04.2._Volterra_dislocation_models.html) for more details on the methods and theory.

## 4. Dislocation anaylsis tools

There are also a number of analysis tools for characterizing dislocations and other crystalline defects.

- **atomman.defect.slip_vector()** calculates the slip vector for all atoms in a system.  This is a good tool for characterizing plastic deformation within a system that occurs between two points in time. 

- **atomman.defect.disregistry()** characterizes a dislocation's planar spreading by calculating the relative displacement of atoms between the atomic planes above and below the dislocation's slip plane. 

- **atomman.defect.nye_tensor()** allows for dislocation cores to be characterized using the gradient of the deformation tensor.  Also calculates the strain state for all atoms in the system relative to a perfect crystal configuration.

- **atomman.defect.differential_displacement()** allows for dislocation cores to be characterized using the difference in the displacements between all pairs of neighbor atoms relative to a perfect crystal configuration.

See the [04.3. Dislocation analysis tools Jupyter Notebook](04.3._Dislocation_analysis_tools.html) for the underlying theory and examples of the tools being used.

## 5. Free surface tools

Atomic configurations with a specific crystalline free surface can be identified using the atomman.defect.free_surface_basis() function.  See the [04.4. Free surface rotation basis Jupyter Notebook](04.4._Free_surface_rotation_basis.html) for the underlying theory and examples of the tool being used.

## 6. Generalized stacking fault / gamma surface tools

The atomman.defect.StackingFault class provides methods for generating atomic configurations of generalized stacking faults that can be used to compute stacking fault energies.  You can generate either a single configuration, or a 1D array of configurations for a slip plane and direction, or a 2D map of configurations exploring the full gamma surface.

See the [04.5. Stacking fault configuration generator Jupyter Notebook](04.5._Stacking_fault_configuration_generator.html) for a description of the class and examples.

The atomman.defect.GammaSurface class provides convenient methods for plotting and interpolating generalized stacking fault ($\gamma$ surface) data.

See the [04.6. Gamma surface plotting Jupyter Notebook](04.6._Gamma_surface_plotting.html) for a description of the class and examples.

## 7. Semidiscrete variational Peierls-Nabarro dislocations

Classes and functions are included that allow for semidiscrete variational Peierls-Nabarro models of dislocations to be solved. 

See the [04.7. Semidiscrete variational Peierls-Nabarro model Jupyter Notebook](04.7._Semidiscrete_variational_Peierls-Nabarro_model.html) for a description of the method and examples. 

## 8. Periodic array of dislocations

The atomman.defect.dislocation_array() function helps generate a periodic array of dislocations atomic configuration useful for studying dislocation slip and dislocations with large stacking faults.

See the [04.8. Periodic array of dislocations Jupyter Notebook](04.8._Periodic_array_of_dislocations.html) for a description of the method and examples. 